# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [4]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [7]:
prod_by_customer = data.groupby(by=['CustomerID', 'ProductName']).Quantity.agg('sum')
pd.DataFrame(prod_by_customer)

Quantity
CustomerID ProductName                                
33         Apricots - Dried                          1
           Assorted Desserts                         1
           Bandage - Flexible Neon                   1
           Bar Mix - Pina Colada, 355 Ml             1
           Beans - Kidney, Canned                    1
           Beef - Chuck, Boneless                    1
           Beef - Prime Rib Aaa                      1
           Beer - Original Organic Lager             1
           Beer - Rickards Red                       1
           Black Currants                            1
           Blackberries                              1
           Bread - Italian Roll With Herbs           1
           Bread Foccacia Whole                      1
           Cake - Box Window 10x10x2.5               1
           Cassis                                    1
           Cheese - Cottage Cheese                   1
           Crab - Imitation Flakes                   1
           Fondant - Icing                           1
           French Pastry - Mini Chocolate            1
           General Purpose Trigger                   1
           Grapes - Red                              1
           Grouper - Fresh                           2
           Hersey Shakes                             1
           Ice Cream Bar - Drumstick                 1
           Lamb - Pieces, Diced                      1
           Lettuce - Spring Mix                      2
           Mangoes                                   1
           Muffin Batt - Choc Chk                    1
           Onions - Vidalia                          1
           Pasta - Cheese / Spinach Bauletti         1
...                                                ...
98200      Onion Powder                             25
           Orange - Canned, Mandarin                25
           Pasta - Detalini, White, Fresh           25
           Puree - Passion Fruit                    25
           Raspberries - Fresh                      25
           Rice - Long Grain                        25
           Rosemary - Dry                           25
           Rosemary - Primerba, Paste               50
           Rum - Coconut, Malibu                    25
           Sage - Ground                            25
           Sausage - Liver                          25
           Sea Bass - Whole                         50
           Seedlings - Mix, Organic                 25
           Sole - Dover, Whole, Fresh               25
           Soup - Campbells, Lentil                 50
           Soup Campbells - Italian Wedding         25
           Soup Knorr Chili With Beans              25
           Soupfoamcont12oz 112con                  25
           Spinach - Baby                           25
           Table Cloth 54x72 White                  25
           Tea - English Breakfast                  25
           Tilapia - Fillets                        25
           Tomatoes Tear Drop                       25
           Truffle Cups - Brown                     25
           Vaccum Bag 10x13                         25
           Vol Au Vents                             50
           Wasabi Powder                            25
           Wine - Fume Blanc Fetzer                 25
           Wine - Hardys Bankside Shiraz            25
           Yogurt - French Vanilla                  25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [8]:
pt_prod_by_customer = pd.pivot_table(data, index='ProductName', 
                                     columns='CustomerID', values='Quantity', aggfunc='sum', fill_value=0)

pt_prod_by_customer


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
Apricots - Halves,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
Apricots Fresh,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
Arizona - Green Tea,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,25,0,0,0,0
Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
from scipy.spatial.distance import pdist, squareform

In [10]:
# 1 / 1 + la distancia, para traducir los valores entre 0 y 1

distances = pd.DataFrame(1/(1 + squareform(pdist(pt_prod_by_customer.T, 'euclidean'))), 
                         index=pt_prod_by_customer.columns, columns=pt_prod_by_customer.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [46]:
top_5_customer_200 = distances.loc[200].sort_values(ascending=False).head(6).reset_index()

lst_users = list(top_5_customer_200.CustomerID[1:])

top_5_customer_200

,CustomerID,200
0,200,1.000000
1,3317,0.084959
2,1072,0.083972
3,3535,0.082403
4,1920,0.082100
5,3909,0.080920


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [43]:
step5 = prod_by_customer.loc[lst_users]
step5

CustomerID  ProductName                      
1072        Appetizer - Mini Egg Roll, Shrimp    1
            Appetizer - Sausage Rolls            1
            Beans - Kidney, Canned               1
            Beef - Montreal Smoked Brisket       1
            Beef - Striploin Aa                  1
            Beef Ground Medium                   1
            Beef Wellington                      1
            Cake - Box Window 10x10x2.5          1
            Cheese - Brie, Triple Creme          1
            Cheese - Parmesan Grated             1
            Chips Potato Salt Vinegar 43g        1
            Coffee - Irish Cream                 1
            Crush - Cream Soda                   1
            Fish - Scallops, Cold Smoked         1
            Foam Dinner Plate                    1
            Fondant - Icing                      1
            Juice - Orange                       1
            Knife Plastic - White                1
            Lentils - Red, Dry      

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [44]:
step5.groupby(['ProductName']).agg('sum')

ProductName
Anchovy Paste - 56 G Tube            1
Appetizer - Mini Egg Roll, Shrimp    1
Appetizer - Sausage Rolls            1
Apricots - Dried                     1
Apricots - Halves                    1
Bacardi Breezer - Tropical           2
Bandage - Fexible 1x3                1
Barramundi                           1
Bay Leaf                             3
Beans - Kidney White                 1
Beans - Kidney, Canned               2
Beef - Chuck, Boneless               1
Beef - Ground Medium                 1
Beef - Montreal Smoked Brisket       1
Beef - Rib Eye Aaa                   1
Beef - Short Loin                    1
Beef - Striploin Aa                  1
Beef - Texas Style Burger            1
Beef - Top Sirloin - Aaa             1
Beef Ground Medium                   1
Beef Wellington                      2
Beer - Blue                          1
Beer - Original Organic Lager        2
Beer - Rickards Red                  1
Beer - Sleemans Cream Ale            1
Berry Brulee 

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.